In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp client

In [ ]:
# export

import os
import time
import httpx
import asyncio
import hashlib

from pathlib import Path

from will_it_saturate.core import Benchmark

<IPython.core.display.Javascript object>

In [ ]:
byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte

benchmark = Benchmark(bandwidth=bandwidth, duration=3)

# file_sizes = [10 ** 7, 10 ** 6]
file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
benchmark.create_rows(file_sizes)

<IPython.core.display.Javascript object>

In [ ]:
# export


class BaseClient:
    def __init__(self, benchmark):
        self.benchmark = benchmark

    def check_md5sums(self, benchmark_files, responses):
        md5_lookup = {}
        for response in responses:
            url = str(response.url)
            md5_lookup[url] = hashlib.md5(response.content).hexdigest()

        for bf in benchmark_files:
            assert bf.md5sum == md5_lookup.get(bf.url, "wrong")


class HttpxClient(BaseClient):
    async def measure_benchmark_row(self, br):
        urls = [bf.url for bf in br.files]
        # httpx breaks on more than 100 parallel connections
        max_connections = min(br.number_of_connections, 100)
        limits = httpx.Limits(
            max_keepalive_connections=5, max_connections=max_connections
        )
        start = time.perf_counter()
        async with httpx.AsyncClient(limits=limits) as client:
            responses = await asyncio.gather(*[client.get(url) for url in urls])
        elapsed = time.perf_counter() - start
        self.check_md5sums(br.files, responses)
        br.elapsed = elapsed

    async def run(self):
        for br in self.benchmark.rows:
            await self.measure_benchmark_row(br)

<IPython.core.display.Javascript object>

In [ ]:
bclient = HttpxClient(benchmark)

<IPython.core.display.Javascript object>

In [ ]:
await bclient.run()

<IPython.core.display.Javascript object>

In [ ]:
for br in benchmark.rows:
    print(br.bytes_per_second)

118178042.79873662
103738741.96430975
59168510.83409966


<IPython.core.display.Javascript object>